<a href="https://colab.research.google.com/github/larajakl/Computational-Linguistics/blob/main/exercises/HomeExercise3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Home Exericse 3: Hyperparameters and Evaluation
In this third home exercise, you will use the knowledge from Tutorial 4 to experiment with hyperparameters, create a test set, and evaluate your final model on the created test set.

In this notebook, please complete all instructions starting with 👋 ⚒ in the code cell after the sign or provide your analysis in the text cell after the sign.

## **Distilbert: Hyperparameters and Evaluation**

Use the code of Tutorial 4 to load and fine-tune the `distilbert-base-cased`model on the small subset of the `imdb`Movie Review Dataset. For convenience, the code of Tutorial 4 required for this exercise is already provided in the code cells below.

👋 ⚒ When creating the dataset splits in the code cell below, additionally create a test set to be used after thet training. Make sure that your test set does not contain any of the sentences contained in the training or validation set and is approximately of the same size as the validation set.

In [ ]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install accelerate --upgrade
!pip install optuna
!pip install optuna-integration[pytorch_lightning]



In [ ]:
from datasets import load_dataset, DatasetDict
from transformers import DataCollatorWithPadding

from transformers import AutoTokenizer

In [ ]:
imdb_dataset = load_dataset("imdb")
# we had loaded the imdb dataset already above - if not, outcomment this line
# Make sure you have the right tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-cased")


# Just take the first n tokens for speed on CPU
def truncate(example):
    return {
        'text': " ".join(example['text'].split()[:100]),
        'label': example['label']
    }

# Random examples for train, validation and test
small_imdb_dataset = DatasetDict(
    train=imdb_dataset['train'].shuffle(seed=24).select(range(128)).map(truncate),
    val=imdb_dataset['train'].shuffle(seed=24).select(range(128, 160)).map(truncate),
    test=imdb_dataset['test'].shuffle(seed=24).select(range(160, 192)).map(truncate)
)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

small_tokenized_dataset = small_imdb_dataset.map(tokenize_function, batched=True, batch_size=16)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

In [ ]:
print(small_imdb_dataset)


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 128
    })
    val: Dataset({
        features: ['text', 'label'],
        num_rows: 32
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 32
    })
})


👋 ⚒ For this exercise, we will use the Hugging Face Trainer class to play with hyperparamters. Try to find a set of hyperparameter settings that achieves the highest possilbe accuracy on the **validation set** with the small dataset and model in this setup.

**Optional:** If you want to follow a more systematic route, feel free to use available frameworks for hyperparameter optimization, such as [Optuna](https://optuna.org/).

In [ ]:
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('distilbert/distilbert-base-cased', num_labels=2)
accuracy = evaluate.load("accuracy")

arguments = TrainingArguments(
    output_dir="sample_cl_trainer-retrain",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=8, # because 8 times 16 is 128
    num_train_epochs=5,
    eval_strategy="epoch", # means it runs validation at the end of each epoch
    save_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    report_to='none',
    seed=224
)

def compute_metrics(eval_pred):
    """Called at the end of validation. Gives accuracy"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # calculates the accuracy
    return accuracy.compute(predictions=predictions, references=labels)


trainer = Trainer(
    model=model,
    args=arguments,
    train_dataset=small_tokenized_dataset['train'],
    eval_dataset=small_tokenized_dataset['val'], # change to test when you do your final evaluation!
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.694600,0.689074,0.468750
2,0.677400,0.687026,0.468750
3,0.659600,0.679423,0.468750
4,0.642500,0.665703,0.687500
5,0.622500,0.661907,0.687500


TrainOutput(global_step=40, training_loss=0.6593048572540283, metrics={'train_runtime': 917.818, 'train_samples_per_second': 0.697, 'train_steps_per_second': 0.044, 'total_flos': 31945341107136.0, 'train_loss': 0.6593048572540283, 'epoch': 5.0})

In [ ]:
# This is how I saved the best model (at best epoch) from this initial manual trial in my Google Drive:
from google.colab import drive
drive.mount('/content/drive')

# Specify model checkpoint directory:
model_checkpoint = "sample_cl_trainer-retrain/checkpoint-40"
# Load the model and tokenizer:
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
# Define the Google Drive directory where you want to save the model:
save_directory = "/content/drive/MyDrive/sample_cl_trainer-retrain_checkpoint-40"
# Save the model and tokenizer to Google Drive:
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


('/content/drive/MyDrive/sample_cl_trainer-retrain_checkpoint-40/tokenizer_config.json',
 '/content/drive/MyDrive/sample_cl_trainer-retrain_checkpoint-40/special_tokens_map.json',
 '/content/drive/MyDrive/sample_cl_trainer-retrain_checkpoint-40/vocab.txt',
 '/content/drive/MyDrive/sample_cl_trainer-retrain_checkpoint-40/added_tokens.json',
 '/content/drive/MyDrive/sample_cl_trainer-retrain_checkpoint-40/tokenizer.json')

In [ ]:
# I used this to check what is actually in my checkpoint-40 folder:
import os
print(os.listdir("sample_cl_trainer/checkpoint-40"))

['training_args.bin', 'model.safetensors', 'scheduler.pt', 'rng_state.pth', 'vocab.txt', 'tokenizer.json', 'trainer_state.json', 'special_tokens_map.json', 'config.json', 'tokenizer_config.json', 'optimizer.pt']


In [ ]:
'''
With this initial set of hyperparameters, I got the best result (in the evaluation on the validation set) at epoch 5,
with 0.6875, as can be seen in the cells above. I saved this version to my Google Drive to be able to come back to it.

Then I used Optuna to test hyperparameters (see below) and I used 15 trials and specified which hyperparameters to tune.
I provided the Optuna study with good baseline hyperparameters that I found in a previous Optuna hyperparameter testing so that the Optuna search
goes in the right direction. Explanation about this first previous Optuna tuning: I had specified a high number of trials and the training
took several hours so I stopped it halfway, but I got some good baseline hyperparameters that I then fed into my Optuna search with 15
trials (visible in the code below).“
'''

In [ ]:
# In the following code cells, I use Optuna to test hyperparameters. New needed imports:
import optuna
import torch

In [ ]:
accuracy = evaluate.load("accuracy")

# Tokenized dataset and data collator are already preloaded as "small_tokenized_dataset" and "data_collator" above

# I use this function to NOT reinitialise the model every time. I only reinitialize layers that are dependent on trial-specific parameters,
# such as the classification head
def reset_classification_head(model, num_labels):
    """
    Reset the classification head of the Hugging Face model
    """
    model.classifier = torch.nn.Linear(model.config.hidden_size, num_labels)
    torch.nn.init.xavier_uniform_(model.classifier.weight)
    torch.nn.init.zeros_(model.classifier.bias)

# Initialize model
model_template = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-cased", num_labels=2)

# Define the objective function
def objective(trial):
    # reset only classification head:
    reset_classification_head(model_template, num_labels=2)
    # Use the updated model_template as the model
    model = model_template
    # Define hyperparameters to tune
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True)
    weight_decay = trial.suggest_float("weight_decay", 0.0, 0.3)
    per_device_train_batch_size = trial.suggest_categorical("per_device_train_batch_size", [8, 16, 32])
    num_train_epochs = trial.suggest_int("num_train_epochs", 3, 10)

    arguments = TrainingArguments(
        output_dir="optuna_cl_trainer",
        per_device_train_batch_size=per_device_train_batch_size,
        per_device_eval_batch_size=16,
        logging_steps=8,
        num_train_epochs=num_train_epochs,
        eval_strategy="epoch",
        save_strategy="epoch",
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        load_best_model_at_end=True,
        report_to="none",
        seed=224
    )

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        return accuracy.compute(predictions=predictions, references=labels)

    trainer = Trainer(
        model=model,
        args=arguments,
        train_dataset=small_tokenized_dataset["train"],
        eval_dataset=small_tokenized_dataset["val"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    # Train the model
    trainer.train()

    # Evaluate the model on validation set
    eval_result = trainer.evaluate()

    # Report intermediate results to Optuna
    trial.report(eval_result["eval_accuracy"], step=0)

    # Check if the trial should be pruned based on evaluation accuracy
    if trial.should_prune():
        raise optuna.exceptions.TrialPruned()

    # Optuna will minimize the return value, so we return the negative of accuracy to maximize it
    return eval_result["eval_accuracy"]

# Create a study to maximize accuracy
study = optuna.create_study(direction="maximize")

# Here I include prior knowledge on good baseline hyperparameters to guide Optuna:
study.enqueue_trial({
    "learning_rate": 3.29432412995289e-05,
    "weight_decay": 0.18368779096914803,
    "per_device_train_batch_size": 32,
    "num_train_epochs": 6
})

# Optimize the study with n_trials
study.optimize(objective, n_trials=15)

# Get the best hyperparameters
print("Best hyperparameters:", study.best_params)



Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[I 2024-11-28 08:47:23,902] A new study created in memory with name: no-name-4334e88d-e879-4386-b772-db9184c94871
<ipython-input-12-8f6dae97a5da>:62: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.685406,0.656250
2,0.677100,0.679872,0.562500
3,0.677100,0.641177,0.750000
4,0.565600,0.624557,0.562500
5,0.565600,0.611920,0.781250
6,0.434700,0.611698,0.750000


[I 2024-11-28 09:03:12,657] Trial 0 finished with value: 0.75 and parameters: {'learning_rate': 3.29432412995289e-05, 'weight_decay': 0.18368779096914803, 'per_device_train_batch_size': 32, 'num_train_epochs': 6}. Best is trial 0 with value: 0.75.
<ipython-input-12-8f6dae97a5da>:62: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.464200,0.718657,0.718750
2,0.105300,0.878886,0.718750
3,0.068300,1.412000,0.750000
4,0.095300,2.445955,0.656250
5,0.087700,1.797547,0.718750
6,0.000200,1.358519,0.750000
7,0.000100,1.400402,0.718750
8,0.000100,1.393377,0.750000
9,0.000100,1.396159,0.750000
10,0.000100,1.398029,0.750000


[I 2024-11-28 09:29:17,401] Trial 1 finished with value: 0.71875 and parameters: {'learning_rate': 4.4126584122243586e-05, 'weight_decay': 0.21686131744562842, 'per_device_train_batch_size': 8, 'num_train_epochs': 10}. Best is trial 0 with value: 0.75.
<ipython-input-12-8f6dae97a5da>:62: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


...

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.865332,0.625000
2,0.052800,0.918469,0.750000
3,0.052800,1.030877,0.750000
4,0.005700,1.082329,0.750000


[I 2024-11-28 12:16:27,503] Trial 13 pruned. 
<ipython-input-12-8f6dae97a5da>:62: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.079700,2.630243,0.687500
2,0.000000,2.110459,0.750000
3,0.000000,2.188514,0.781250
4,0.000000,2.224112,0.781250
5,0.000000,2.239320,0.781250


[I 2024-11-28 12:28:06,463] Trial 14 finished with value: 0.75 and parameters: {'learning_rate': 3.562367373473637e-05, 'weight_decay': 0.24789977789775125, 'per_device_train_batch_size': 8, 'num_train_epochs': 5}. Best is trial 0 with value: 0.75.


Best hyperparameters: {'learning_rate': 3.29432412995289e-05, 'weight_decay': 0.18368779096914803, 'per_device_train_batch_size': 32, 'num_train_epochs': 6}


In [ ]:
'''
The best hyperparameter set that resulted from the above Optuna tuning is actually the one that I fed into the Optuna study as a guideline.
This indicates that Optuna did not find any better hyperparameters within the 15 trials.
In the following, I train a final model using these best hyperparameters. I unfortunately did not immediately save the best model resulting from
the Optuna hyperparameter tuning above, but I did save the best hyperparameters:
'''
best_params = {'learning_rate': 3.29432412995289e-05, 'weight_decay': 0.18368779096914803, 'per_device_train_batch_size': 32, 'num_train_epochs': 6}

final_model = AutoModelForSequenceClassification.from_pretrained("distilbert/distilbert-base-cased", num_labels=2)

# Now I am training this model with these hyperparameters:
final_arguments = TrainingArguments(
    output_dir="final_cl_trainer-retrain",
    per_device_train_batch_size=best_params["per_device_train_batch_size"],
    per_device_eval_batch_size=16,
    logging_steps=8,
    num_train_epochs=best_params["num_train_epochs"],
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=best_params["learning_rate"],
    weight_decay=best_params["weight_decay"],
    load_best_model_at_end=True,
    report_to='none',
    seed=224
)

final_trainer = Trainer(
    model=final_model,
    args=final_arguments,
    train_dataset=small_tokenized_dataset['train'],
    eval_dataset=small_tokenized_dataset['val'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Train the final model
final_trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-24-2fb78188a27d>:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  final_trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.689698,0.468750
2,0.684800,0.695877,0.468750
3,0.684800,0.678666,0.500000
4,0.641600,0.652112,0.718750
5,0.641600,0.640769,0.781250
6,0.573800,0.636286,0.812500


TrainOutput(global_step=24, training_loss=0.6334226727485657, metrics={'train_runtime': 1113.5333, 'train_samples_per_second': 0.69, 'train_steps_per_second': 0.022, 'total_flos': 38349311910912.0, 'train_loss': 0.6334226727485657, 'epoch': 6.0})

In [ ]:
# Specify model checkpoint directory:
model_checkpoint_final = "final_cl_trainer-retrain/checkpoint-24"
# Load the model and tokenizer:
final_model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint_final)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint_final)
# Define the Google Drive directory where you want to save the model:
save_directory = "/content/drive/MyDrive/final_cl_trainer-retrain_checkpoint-24"
# Save the model and tokenizer to Google Drive:
final_model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

('/content/drive/MyDrive/final_cl_trainer-retrain_checkpoint-24/tokenizer_config.json',
 '/content/drive/MyDrive/final_cl_trainer-retrain_checkpoint-24/special_tokens_map.json',
 '/content/drive/MyDrive/final_cl_trainer-retrain_checkpoint-24/vocab.txt',
 '/content/drive/MyDrive/final_cl_trainer-retrain_checkpoint-24/added_tokens.json',
 '/content/drive/MyDrive/final_cl_trainer-retrain_checkpoint-24/tokenizer.json')

👋 ⚒ Change the following code cell in a way that not only a single sentence is evaluated on your trained model (!make sure to use the correct checkpoint!) but the evaluation is performaned on the entire newly created test set.

This might also be a good occassion to get familiar with the [Hugging Face documentation and tutorials](https://huggingface.co/docs/transformers/index).

In [ ]:
# This just tests 1 sentence and gives me prediction:
test_str = "I don't like but also don't dislike this movie!"
model_inputs = tokenizer(test_str, return_tensors="pt")

best_model_Optuna = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/final_cl_trainer-retrain_checkpoint-24")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/final_cl_trainer-retrain_checkpoint-24")

prediction_Optuna = torch.argmax(best_model_Optuna(**model_inputs).logits)
print(["NEGATIVE", "POSITIVE"][prediction_Optuna])




NEGATIVE


In [ ]:
# Testing final trained model on entire test set:

best_model_Optuna = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/final_cl_trainer-retrain_checkpoint-24")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/final_cl_trainer-retrain_checkpoint-24")

arguments_for_testing = TrainingArguments(
    output_dir="evaluation_on_test_set",
    per_device_eval_batch_size=16,
    report_to='none',
    seed=224
)

trainer_for_testing = Trainer(
    model=best_model_Optuna,
    args=arguments_for_testing,
    eval_dataset=small_tokenized_dataset['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

test_results_best_model_Optuna = trainer_for_testing.evaluate()
print("Test results of the best model tuned by Optuna:", test_results_best_model_Optuna)

<ipython-input-30-371ea2a31834>:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_for_testing = Trainer(


Test results of the best model tuned by Optuna: {'eval_loss': 0.6482808589935303, 'eval_model_preparation_time': 0.0029, 'eval_accuracy': 0.6875, 'eval_runtime': 12.3508, 'eval_samples_per_second': 2.591, 'eval_steps_per_second': 0.162}


In [ ]:
'''
In conclusion, the accuracy of the best model resulting from the Optuna hyperparameter tuning amounts to 0.6875 (on the test set). This is quite disappointing,
especially since we learned in the machine learning class that the metric accuracy is not even the best metric to use (it doesn't take all parts of
the confusion matrix into account).
The reason why the accuracy is very low can be the result of the following facts:
- The dataset I used is small (I only used 128 samples for train set and 32 for validation and 32 for test) - I wanted to use bigger sets but my
computer's capacities are quite limited so I used the initial ones.
- I set the truncation to 100; a higher value could potentially improve results (but more padding might also affect the model).
- I split the data into train/val/test in a way that is not ideal and hence the 3 sets might not be perfectly separated. The code technically
works because I indicate ranges, but I apply shuffling 3 times and not knowing how the shuffling really works internally, this can make the model a bit
unpredictable and it is probably not the cleanest way of handling this. For next time, it might be better to shuffle once and then split into the 3 sets.
- The accuracy when evaluated on the validation set is significantly higher (0.8125) than the accuracy on the test set (0.6875). Hence, the model
might be overfitted on the validation set because the hyperparameters were fine-tuned to the validation set (-> overfitting to validation set can happen
especially in cases with small datasets such as my case).

If I had had more time, I would have continued tuning hyperparameters for a more desirable result and I would have used
cross-validation just in case to not overfit the tuning to the validation set. Since the Optuna tuning took hours (even for
the 15 trials and the small datasets) and Google Colab disconnects after a while, I would continue with manual tuning from here,
and potentially try to play with truncation a bit more.
'''